In [1]:
!apt-get install libomp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (1,095 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

In [10]:
%%writefile merge.cpp
#include <iostream>
#include <omp.h>

using namespace std;

void merge(int arr[], int l, int m, int r) {
    int n1 = m - l + 1;
    int n2 = r - m;

    int L[n1], R[n2];

    for (int i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (int j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    int i = 0;
    int j = 0;
    int k = l;
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }
}

void mergeSortSerial(int arr[], int l, int r) {
    if (l >= r) {
        return;
    }
    int m = l + (r - l) / 2;

    mergeSortSerial(arr, l, m);
    mergeSortSerial(arr, m + 1, r);

    merge(arr, l, m, r);
}

void mergeSortParallel(int arr[], int l, int r) {
    if (l >= r) {
        return;
    }
    int m = l + (r - l) / 2;

    #pragma omp parallel sections
    {
        #pragma omp section
        mergeSortParallel(arr, l, m);
        #pragma omp section
        mergeSortParallel(arr, m + 1, r);
    }
    merge(arr, l, m, r);
}

int main() {
    int n;
    cout << "Enter the number of elements: ";
    cin >> n;

    int arr_serial[n], arr_parallel[n];
    cout << "Enter the elements: ";
    for (int i = 0; i < n; i++) {
        cin >> arr_serial[i];
        arr_parallel[i] = arr_serial[i];
    }

    double start, end;

    // Measure performance of serial merge sort
    start = omp_get_wtime();
    mergeSortSerial(arr_serial, 0, n - 1);
    end = omp_get_wtime();
    cout << "Sorted array using serial merge sort: ";
    for (int i = 0; i < n; i++) {
        cout << arr_serial[i] << " ";
    }
    cout << endl;
    cout << "Serial merge sort time: " << end - start << " seconds" << endl;

    // Measure performance of parallel merge sort
    start = omp_get_wtime();
    mergeSortParallel(arr_parallel, 0, n - 1);
    end = omp_get_wtime();
    cout << "Sorted array using parallel merge sort: ";
    for (int i = 0; i < n; i++) {
        cout << arr_parallel[i] << " ";
    }
    cout << endl;
    cout << "Parallel merge sort time: " << end - start << " seconds" << endl;

    return 0;
}


Writing merge.cpp


In [11]:
!clang++ -fopenmp merge.cpp -o merge
!./merge

Enter the number of elements: 4
Enter the elements: 121 33 45 17
Sorted array using serial merge sort: 17 33 45 121 
Serial merge sort time: 4.05312e-06 seconds
Sorted array using parallel merge sort: 17 33 45 121 
Parallel merge sort time: 0.00130987 seconds
